In [1]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 4.4 MB/s eta 0:00:00


In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.3 MB/s eta 0:00:00


In [3]:
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_dataset
import evaluate
import numpy as np
from torch.utils.data import Dataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
source_lang = "ko"
target_lang = "en"
dataset_name = f"iwslt2017-{source_lang}-{target_lang}"
dataset = load_dataset("iwslt2017", dataset_name)
print(dataset['train'][0])

iwslt2017.py:   0%|          | 0.00/8.17k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/18.5k [00:00<?, ?B/s]

The repository for iwslt2017 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/iwslt2017.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


ko-en.zip:   0%|          | 0.00/19.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/230240 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8514 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/879 [00:00<?, ? examples/s]

{'translation': {'en': 'Thank you so much, Chris.', 'ko': '감사합니다, 크리스. 이곳에 두 번이나'}}


In [5]:
from transformers import MBart50Tokenizer

tokenizer = MBart50Tokenizer.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

print(tokenizer.lang_code_to_id)

source_lang = 'en_XX'
target_lang = 'ko_KR'

tokenizer.src_lang = source_lang
tokenizer.tgt_lang = target_lang

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


{'ar_AR': 250001, 'cs_CZ': 250002, 'de_DE': 250003, 'en_XX': 250004, 'es_XX': 250005, 'et_EE': 250006, 'fi_FI': 250007, 'fr_XX': 250008, 'gu_IN': 250009, 'hi_IN': 250010, 'it_IT': 250011, 'ja_XX': 250012, 'kk_KZ': 250013, 'ko_KR': 250014, 'lt_LT': 250015, 'lv_LV': 250016, 'my_MM': 250017, 'ne_NP': 250018, 'nl_XX': 250019, 'ro_RO': 250020, 'ru_RU': 250021, 'si_LK': 250022, 'tr_TR': 250023, 'vi_VN': 250024, 'zh_CN': 250025, 'af_ZA': 250026, 'az_AZ': 250027, 'bn_IN': 250028, 'fa_IR': 250029, 'he_IL': 250030, 'hr_HR': 250031, 'id_ID': 250032, 'ka_GE': 250033, 'km_KH': 250034, 'mk_MK': 250035, 'ml_IN': 250036, 'mn_MN': 250037, 'mr_IN': 250038, 'pl_PL': 250039, 'ps_AF': 250040, 'pt_XX': 250041, 'sv_SE': 250042, 'sw_KE': 250043, 'ta_IN': 250044, 'te_IN': 250045, 'th_TH': 250046, 'tl_XX': 250047, 'uk_UA': 250048, 'ur_PK': 250049, 'xh_ZA': 250050, 'gl_ES': 250051, 'sl_SI': 250052}


In [6]:
print(dataset['train'])
print(dataset['train'].column_names)
print(dataset['train'][0])

Dataset({
    features: ['translation'],
    num_rows: 230240
})
['translation']
{'translation': {'en': 'Thank you so much, Chris.', 'ko': '감사합니다, 크리스. 이곳에 두 번이나'}}


In [7]:
def preprocess_function(examples):
    inputs = [ex['en'] for ex in examples['translation']]
    targets = [ex['ko'] for ex in examples['translation']]

    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding='max_length')

    labels = tokenizer(targets, max_length=128, truncation=True, padding='max_length')

    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

print(tokenized_datasets['train'].column_names)

Map:   0%|          | 0/230240 [00:00<?, ? examples/s]

Map:   0%|          | 0/8514 [00:00<?, ? examples/s]

Map:   0%|          | 0/879 [00:00<?, ? examples/s]

['translation', 'input_ids', 'attention_mask', 'labels']


In [8]:
def preprocess_function(examples):
    inputs = [ex['en'] for ex in examples['translation']]
    targets = [ex['ko'] for ex in examples['translation']]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)
    labels = tokenizer(targets, max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(
    lambda batch: tokenizer(
        [example['en'] for example in batch['translation']],
        padding='max_length',
        truncation=True,
        max_length=128
    ),
    batched=True
)

Map:   0%|          | 0/230240 [00:00<?, ? examples/s]

Map:   0%|          | 0/8514 [00:00<?, ? examples/s]

Map:   0%|          | 0/879 [00:00<?, ? examples/s]

In [9]:
class TranslationDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, translations):
        self.encodings = encodings
        self.translations = translations

    def __getitem__(self, idx):
        item = {
            "input_ids": torch.tensor(self.encodings["input_ids"][idx]),
            "attention_mask": torch.tensor(self.encodings["attention_mask"][idx]),
            "labels": torch.tensor(self.translations["input_ids"][idx])
        }
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_encodings = tokenized_datasets["train"]
train_translations = tokenized_datasets["train"]

train_dataset = TranslationDataset(train_encodings, train_translations)
val_dataset = TranslationDataset(tokenized_datasets["validation"], tokenized_datasets["validation"])
test_dataset = TranslationDataset(tokenized_datasets["test"], tokenized_datasets["test"])

In [12]:
from transformers import MBartForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments

model_name = "facebook/mbart-large-50"

device = "cuda" if torch.cuda.is_available() else "cpu"
model = MBartForConditionalGeneration.from_pretrained(model_name).to(device)

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

metric = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_labels = [[label] for label in decoded_labels]
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print(tokenized_datasets)


/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


DatasetDict({
    train: Dataset({
        features: ['translation', 'input_ids', 'attention_mask'],
        num_rows: 230240
    })
    test: Dataset({
        features: ['translation', 'input_ids', 'attention_mask'],
        num_rows: 8514
    })
    validation: Dataset({
        features: ['translation', 'input_ids', 'attention_mask'],
        num_rows: 879
    })
})


In [13]:
print(tokenized_datasets.column_names)

{'train': ['translation', 'input_ids', 'attention_mask'], 'test': ['translation', 'input_ids', 'attention_mask'], 'validation': ['translation', 'input_ids', 'attention_mask']}


In [14]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds

    preds = torch.tensor(preds).to("cuda" if torch.cuda.is_available() else "cpu")
    labels = torch.tensor(labels).to("cuda" if torch.cuda.is_available() else "cpu")

    preds = torch.where(preds != -100, preds, tokenizer.pad_token_id)
    labels = torch.where(labels != -100, labels, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(preds.tolist(), skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels.tolist(), skip_special_tokens=True)

    decoded_labels = [[label] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)

    return {"test_bleu": result["score"]}

test_results = trainer.evaluate(eval_dataset=test_dataset, metric_key_prefix="test")
print("Test BLEU score:", test_results["test_bleu"])

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Test BLEU score: 88.89412604743973


In [ ]:
import torch
torch.cuda.empty_cache()